# **Some thoughts on dbo (database owner)**

Fabiano Neves Amorim - amorim@pythian.com

Starting with a question. **Who should own a database?**

By own, I mean the server principal (login) that owns the database, in other words, the login mapped to the dbo (database owner) user in a database.

By default, the owner of a DB is the one that issued the create/restore database statement. Also it is important to remeber that this login will be automatically granted as db_owner database role in the DB.

So, that means an old employee that created the DB 10 years ago, is probably the owner of the DB.

But, the question is, is this ok have this user has a db owner?

## Demo using a SQL login has db owner

In [ ]:
USE master
GO

-- Let's start by creating a dbTest1 db.
DROP DATABASE IF EXISTS dbTest1
CREATE DATABASE dbTest1
GO

At this time, login used to create the DB is the db owner

In [2]:
SELECT name AS dbName,
     SUSER_SNAME(owner_sid) UserName,
     IS_SRVROLEMEMBER('sysadmin', SUSER_SNAME(owner_sid)) is_sysadmin
FROM sys.databases
WHERE name = 'dbTest1';
GO

(1 row affected)

Total execution time: 00:00:00.090

dbName,UserName,is_sysadmin
dbTest1,RAZERFABIANO\Fabiano,1


Creating a couple of logins we'll use in the demo

In [ ]:
IF EXISTS(SELECT * FROM sys.sql_logins WHERE name = 'TestLogin1')
BEGIN
  DROP LOGIN TestLogin1;
  DROP LOGIN TestLogin2;
END
GO
CREATE LOGIN TestLogin1 WITH PASSWORD = N'102030'
CREATE LOGIN TestLogin2 WITH PASSWORD = N'102030'
GO

Set dbo to TestLogin1

In [ ]:
ALTER AUTHORIZATION ON DATABASE::dbTest1 TO TestLogin1;
GO

Creating function to map Users to Role

In [ ]:
USE dbTest1
GO
CREATE OR ALTER FUNCTION dbo.dbRolesUsersMap (@dbRole SYSNAME = '%')
RETURNS TABLE
AS
RETURN (
    SELECT 
      User_Type = 
         CASE mmbrp.[type] 
         WHEN 'G' THEN 'Windows Group' 
         WHEN 'S' THEN 'SQL User' 
         WHEN 'U' THEN 'Windows User' 
         END,
       Database_User_Name = mmbrp.[name],
       Login_Name = ul.[name],
       DB_Role = rolp.[name]
    FROM sys.database_role_members mmbr, -- The Role OR members associations table
       sys.database_principals rolp,     -- The DB Roles names table
       sys.database_principals mmbrp,    -- The Role members table (database users)
       sys.server_principals ul          -- The Login accounts table
    WHERE Upper (mmbrp.[type]) IN ( 'S', 'U', 'G' )
       -- No need for these system account types
       AND Upper (mmbrp.[name]) NOT IN ('SYS','INFORMATION_SCHEMA')
       AND rolp.[principal_id] = mmbr.[role_principal_id]
       AND mmbrp.[principal_id] = mmbr.[member_principal_id]
       AND ul.[sid] = mmbrp.[sid]
       AND rolp.[name] LIKE '%' + @dbRole + '%'
    )
GO

As we can see, login TestLogin1 has db_owner on dbTest1

In [6]:
SELECT * FROM dbTest1.dbo.dbRolesUsersMap (DEFAULT)
GO

(1 row affected)

Total execution time: 00:00:00.028

User_Type,Database_User_Name,Login_Name,DB_Role
SQL User,dbo,TestLogin1,db_owner


At this time, we can't drop this user because it is owner of dbTest1...

In [ ]:
DROP LOGIN TestLogin1;
GO
--Msg 15174, Level 16, State 1, Line 75
--Login 'TestLogin1' owns one or more database(s). Change the owner of the database(s) before dropping the login.


Create a test table

In [ ]:
USE dbTest1
DROP TABLE IF EXISTS TableTest1
CREATE TABLE TableTest1 (Col1 INT);
GO

dbo (TestLogin1) user can do anything on DB ... 

In [9]:
USE master
EXECUTE AS LOGIN = 'TestLogin1';
SELECT USER_NAME(), ORIGINAL_LOGIN(); -- Check user context
GO
SELECT Col1 FROM dbTest1.dbo.TableTest1;
GO
REVERT;
GO

(1 row affected)

(0 rows affected)

Commands completed successfully.

Total execution time: 00:00:00.008

Col1


(No column name),(No column name)
guest,RAZERFABIANO\Fabiano


TestLogin2 is not allowed to access to dbTest1...

In [10]:
USE master
EXECUTE AS LOGIN = 'TestLogin2';
SELECT USER_NAME(), ORIGINAL_LOGIN(); -- Check user context
GO
SELECT Col1 FROM dbTest1.dbo.TableTest1;
--Msg 916, Level 14, State 2, Line 105
--The server principal "TestLogin2" is not able to access the database "dbTest1" under the current security context.
GO
REVERT;
GO

(1 row affected)

: Msg 916, Level 14, State 2, Line 5
The server principal "TestLogin2" is not able to access the database "dbTest1" under the current security context.

Commands completed successfully.

Total execution time: 00:00:00.006

(No column name),(No column name)
guest,RAZERFABIANO\Fabiano


First lesson, application shouldn't connect using db owner login because this has a lot more privileges that are necessary.

A good practice would be disable this user to guarantee no one is logging with it.

**Note:** Users can still use a disabled user with "EXECUTE AS"...

## Demo using a Windows Account has db owner

An windows account looks fine, but what happens when this account is removed? 

Would this be a problem?

In [11]:
USE master
GO
-- Creating a Win account we'll use in this test...
EXEC xp_cmdshell 'NET USER WinAccountLogin1 /DELETE'
EXEC xp_cmdshell 'NET USER WinAccountLogin1 102030 /ADD'
GO

Commands completed successfully.

(3 rows affected)

(3 rows affected)

Total execution time: 00:00:00.893

output
The command completed successfully.
NULL
NULL


output
The command completed successfully.
NULL
NULL


Let's start by creating a dbTest1 db.

In [ ]:
DROP DATABASE IF EXISTS dbTest1
CREATE DATABASE dbTest1
GO

Create a SQL Login for this account

In [ ]:
IF EXISTS(SELECT * FROM sys.server_principals WHERE name = 'RAZERFABIANO\WinAccountLogin1')
  DROP LOGIN [RAZERFABIANO\WinAccountLogin1]
GO
CREATE LOGIN [RAZERFABIANO\WinAccountLogin1] FROM WINDOWS
GO

Set dbo to [RAZERFABIANO\WinAccountLogin1]

In [ ]:
ALTER AUTHORIZATION ON DATABASE::dbTest1 TO [RAZERFABIANO\WinAccountLogin1];
GO

Creating a table to do some tests

In [ ]:
USE dbTest1
DROP TABLE IF EXISTS TableTest1
CREATE TABLE TableTest1 (Col1 INT);
GO

Creating a proc to run as owner

In [ ]:
DROP PROC IF EXISTS Proc1
GO
CREATE PROC Proc1
WITH EXECUTE AS OWNER
AS
SELECT USER_NAME(), ORIGINAL_LOGIN(); -- Check user context
SELECT * FROM TableTest1
GO

Creating a login with access only to connect into the DB and run the SP...

In [ ]:
IF EXISTS(SELECT * FROM sys.sql_logins WHERE name = 'TestLogin1')
  DROP LOGIN TestLogin1;
GO
CREATE LOGIN TestLogin1 WITH PASSWORD = N'102030'
GO

Grant DB access and EXEC on Proc1

In [ ]:
USE dbTest1
CREATE USER TestLogin1 FOR LOGIN TestLogin1
GRANT EXECUTE ON OBJECT::dbo.Proc1 TO TestLogin1;  
GO

If I try to run the SP using TestLogin1 it should work

In [19]:
EXECUTE AS LOGIN = 'TestLogin1';
GO
EXEC dbTest1.dbo.Proc1
GO
REVERT
GO

Commands completed successfully.

(1 row affected)

(0 rows affected)

Commands completed successfully.

Total execution time: 00:00:00.010

(No column name),(No column name)
dbo,RAZERFABIANO\Fabiano


Col1


Let's remove Windows user

In [20]:
EXEC xp_cmdshell 'NET USER WinAccountLogin1 /DELETE'
GO

(3 rows affected)

Total execution time: 00:00:00.395

output
The command completed successfully.
NULL
NULL


Now EXECUTE AS proc will fail...

In [21]:
EXECUTE AS LOGIN = 'TestLogin1';
GO
EXEC dbTest1.dbo.Proc1
GO
REVERT
GO
--Msg 15404, Level 16, State 11, Procedure dbTest1.dbo.Proc1, Line 0 [Batch Start Line 216]
--Could not obtain information about Windows NT group/user 'RAZERFABIANO\WinAccountLogin1', error code 0x534.

Commands completed successfully.

: Msg 15404, Level 16, State 11, Procedure dbTest1.dbo.Proc1, Line 0
Could not obtain information about Windows NT group/user 'RAZERFABIANO\WinAccountLogin1', error code 0x534.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.001

Add user again will "fix" it...

In [22]:
EXEC xp_cmdshell 'NET USER WinAccountLogin1 102030 /ADD'
GO

(3 rows affected)

Total execution time: 00:00:00.370

output
The command completed successfully.
NULL
NULL


 Now it works...

In [23]:
EXECUTE AS LOGIN = 'TestLogin1';
GO
EXEC dbTest1.dbo.Proc1
GO
REVERT
GO

Commands completed successfully.

(1 row affected)

(0 rows affected)

Commands completed successfully.

Total execution time: 00:00:00.009

(No column name),(No column name)
RAZERFABIANO\WinAccountLogin1,RAZERFABIANO\Fabiano


Col1


Let's create a DB backup to restore after we drop the Login...

In [ ]:
USE master
GO
EXEC xp_cmdshell 'del C:\temp\dbTest1.bak'
BACKUP DATABASE [dbTest1] TO  DISK = N'C:\temp\dbTest1.bak' 
WITH NOFORMAT, NOINIT,  NAME = N'dbTest1-Full Database Backup', 
SKIP, NOREWIND, NOUNLOAD, COMPRESSION,  STATS = 10
GO
DROP DATABASE IF EXISTS dbTest1
GO

Removing Login RAZERFABIANO\WinAccountLogin1
To simulate a scenario where DB will be restored in a new Instance 
where user was not created

In [ ]:
DROP LOGIN [RAZERFABIANO\WinAccountLogin1];
GO

What will happen if we restore the DB? 

In [ ]:
RESTORE DATABASE [dbTest1] FROM  DISK = N'c:\temp\dbTest1.bak' 
WITH  FILE = 1,  NOUNLOAD,  STATS = 5
GO

Now it fails because logging specified as db owner doesn't exist

In [27]:
EXECUTE AS LOGIN = 'TestLogin1';
GO
EXEC dbTest1.dbo.Proc1
GO
REVERT
GO
-- Msg 15517, Level 16, State 1, Procedure dbTest1.dbo.Proc1, Line 0 [Batch Start Line 264]
-- Cannot execute as the database principal because the principal "dbo" does not exist, 
-- this type of principal cannot be impersonated, or you do not have permission.

Commands completed successfully.

: Msg 15517, Level 16, State 1, Procedure dbTest1.dbo.Proc1, Line 0
Cannot execute as the database principal because the principal "dbo" does not exist, this type of principal cannot be impersonated, or you do not have permission.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.004

But, wait, when I restored the DB shouldn't it change the owner to the user running the 
RESTORE DATABASE command? 

Let's check who is the DB owner

In [28]:
-- Owner according to master is "RAZERFABIANO\Fabiano", that is the 
-- user that ran RESTORE DATABASE command
SELECT SUSER_SNAME(owner_sid), owner_sid FROM sys.databases WHERE name = 'dbTest1'

-- Owner according to database is unknown as sid doesn't map to any existing login
-- Classic orphaned user case, right?...
SELECT SUSER_SNAME(sid), sid FROM dbTest1.sys.database_principals WHERE name = 'dbo'
GO

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.041

(No column name),owner_sid
RAZERFABIANO\Fabiano,0x01050000000000051500000067690A979A5095932F9AF370E9030000


(No column name),sid
NULL,0x01050000000000051500000067690A979A5095932F9AF37016040000


Let's check the orphaned users report

In [29]:
USE dbTest1
EXEC sp_change_users_login 'Report'
GO
-- Ops no results, yep, that's right, there is no RAZERFABIANO\WinAccountLogin1 user

Commands completed successfully.

UserName,UserSID


Commands completed successfully.

Total execution time: 00:00:00.053

Ok, what if we recreate the Login?

In [ ]:
CREATE LOGIN [RAZERFABIANO\WinAccountLogin1] FROM WINDOWS
GO

Still no luck

In [31]:
EXECUTE AS LOGIN = 'TestLogin1';
GO
EXEC dbTest1.dbo.Proc1
GO
REVERT
GO

Commands completed successfully.

: Msg 15517, Level 16, State 1, Procedure dbTest1.dbo.Proc1, Line 0
Cannot execute as the database principal because the principal "dbo" does not exist, this type of principal cannot be impersonated, or you do not have permission.

Commands completed successfully.

Total execution time: 00:00:00.002

That's because SID of new login is different that SID on database

In [32]:
SELECT name, sid FROM sys.server_principals WHERE name = 'RAZERFABIANO\WinAccountLogin1'
-- 0x01050000000000051500000067690A979A5095932F9AF37015040000
GO
SELECT SUSER_SNAME(sid), sid FROM dbTest1.sys.database_principals WHERE name = 'dbo'
-- 0x01050000000000051500000067690A979A5095932F9AF37014040000
GO

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.020

name,sid
RAZERFABIANO\WinAccountLogin1,0x01050000000000051500000067690A979A5095932F9AF37017040000


(No column name),sid
NULL,0x01050000000000051500000067690A979A5095932F9AF37016040000


If we to set owner back to [RAZERFABIANO\WinAccountLogin1] it will update the SID and fix the issue...

In [ ]:
ALTER AUTHORIZATION ON DATABASE::dbTest1 TO [RAZERFABIANO\WinAccountLogin1];
GO

Now it works...

In [34]:
EXECUTE AS LOGIN = 'TestLogin1';
GO
EXEC dbTest1.dbo.Proc1
GO
REVERT
GO

Commands completed successfully.

(1 row affected)

(0 rows affected)

Commands completed successfully.

Total execution time: 00:00:00.010

(No column name),(No column name)
dbo,RAZERFABIANO\Fabiano


Col1


So, what people do to avoid those problems? 
Easy, set db owner to sa.
  
But, is this really a good option?
I understand it is "easier" from a maintenance perspective to use sa has the owner, but there is a risk of elevation of privilege and it could be worse in case the DB is set to trustworthy.

From that perspective, **a risk is a risk** and we need to honor the basic **PoLP**(**P**rinciple of **L**east **P**rivilege).
Means, an user must be able to access and should only have privileges to information/resources that are necessary for its legitimate purpose.

## Demo, using SA has db owner

Remember, usually an attack starts from inside, an employee not happy with latest bonus or with bad intention may decide to do things he shouldn't be able to do unless sysadmin privileged is available... Well, it really doesn’t matter whether user has sysadmin role or even if the sa account is disabled.
If sa owns a DB, an attacker could escalate privilege to sysadmin...

Let's see a sample...

Let's consider a scenario where you've a Junior DBA or an application login with db_owner role in a DB.


Let's start by creating a dbTest1 db.

Notice that I'm setting DB to trustworthy, this is required to escalate the privilege.  

In [ ]:
USE master
GO
DROP DATABASE IF EXISTS dbTest1
CREATE DATABASE dbTest1
ALTER DATABASE dbTest1 SET TRUSTWORTHY ON
GO

Creating a login we'll use in the demo

In [ ]:
IF EXISTS(SELECT * FROM sys.sql_logins WHERE name = 'TestLogin1')
DROP LOGIN TestLogin1;
GO
CREATE LOGIN TestLogin1 WITH PASSWORD = N'102030'
GO

Set dbo to sa

In [ ]:
ALTER AUTHORIZATION ON DATABASE::dbTest1 TO sa;
GO

Grant TestLogin1 login DB access and db_ddladmin role

In [ ]:
USE dbTest1
DROP USER IF EXISTS TestLogin1
CREATE USER TestLogin1 FOR LOGIN TestLogin1;
GO
EXEC sp_addrolemember @rolename = 'db_owner', @membername = 'TestLogin1';
GO

What if I try to run xp_cmdshell to create a local admin user?
User should not have access to xp_cmdshell and will receive permission denied message...

In [40]:
EXECUTE AS LOGIN = 'TestLogin1';
SELECT USER_NAME(), ORIGINAL_LOGIN(); -- Check user context
EXEC xp_cmdshell 'echo "Hello world"'
GO
REVERT;
GO
--Msg 229, Level 14, State 5, Procedure xp_cmdshell, Line 1 [Batch Start Line 382]
--The EXECUTE permission was denied on the object 'xp_cmdshell', database 'mssqlsystemresource', schema 'sys'.

(1 row affected)

: Msg 229, Level 14, State 5, Procedure xp_cmdshell, Line 1
The EXECUTE permission was denied on the object 'xp_cmdshell', database 'mssqlsystemresource', schema 'sys'.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.006

(No column name),(No column name)
TestLogin1,RAZERFABIANO\Fabiano


But I can create a procedure... right? I'm a db_owner...
So, let's create a special proc

In [ ]:
EXECUTE AS LOGIN = 'TestLogin1';
DROP PROC IF EXISTS my_xp_cmdshell
GO
CREATE PROC my_xp_cmdshell @Str VARCHAR(8000)
WITH EXECUTE AS OWNER
AS
EXEC xp_cmdshell @Str
GO
REVERT;
GO

Now I've my own xp_cmdshell version...

In [42]:
EXECUTE AS LOGIN = 'TestLogin1';
EXEC my_xp_cmdshell @Str = 'echo "Hello world"'
GO
REVERT;
GO

(2 rows affected)

Commands completed successfully.

Total execution time: 00:00:00.259

output
"""Hello world"""
NULL


## Final recommendations:

- As a best practice, we should use either a Windows Account or a disabled 
SQL Login with very minimum permission to minimize risks of priv. escalation, 
even in scenarios where a DB is set to trustworthy. An instance may not be vulnerably at the time we check, but as soon as someone (a Dev trying to create an unsafe CLR?...) set a DB to trustworthy...

- Disable the trustworthy from all (except MSDB) databases. If you need to use functionality that is controlled by the trustworthy option, use digital signatures to enable the functionality.

Links:

https://support.microsoft.com/en-us/help/2183687/guidelines-for-using-the-trustworthy-database-setting-in-sql-server

https://docs.microsoft.com/en-us/archive/blogs/sqlsecurity/the-trustworhy-bit-database-property-in-sql-server-2005
